TODO:
1. convert file to PDF if it is not PDF
2. take the file and scrap by each page as a markdown table
3. Convert with the tool markdown to JSON
4. Prepare JSON to publish to gsheet
5. Publish to gsheet

In [ ]:
import os
from dotenv import load_dotenv

# load .env
load_dotenv(dotenv_path='../.env')

# get API key
api_key = os.getenv('ANTHROPIC_API_KEY')

print("api_key -> ", api_key)

In [ ]:
from anthropic import Anthropic

client = Anthropic(api_key=api_key)

In [ ]:
from PyPDF2 import PdfReader, PdfWriter
from io import BytesIO
import base64
from pprint import pp

def curr_page_pdf(pdf_path: str, page_num: int) -> str:
    reader = PdfReader(pdf_path)
    
    # Проверяем количество страниц и валидность номера страницы
    total_pages = len(reader.pages)
    if page_num < 0 or page_num >= total_pages:
        raise ValueError(f"Номер страницы должен быть от 0 до {total_pages-1}")
        
    # Создаем новый PDF только с нужной страницей
    writer = PdfWriter()
    writer.add_page(reader.pages[page_num])
    
    # Сохраняем в байтовый поток
    output = BytesIO()
    writer.write(output)
    
    # Конвертируем в base64
    pdf_bytes = output.getvalue()
    base64_pdf = base64.b64encode(pdf_bytes).decode('utf-8')
    
    output.close()
    return base64_pdf


In [ ]:
def extract_text_from_pdf(pdf_data):
    response = client.beta.messages.create(
        model="claude-3-5-sonnet-20241022",
        betas=["pdfs-2024-09-25"],
        max_tokens=8164,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "document",
                        "source": {
                            "type": "base64",
                            "media_type": "application/pdf",
                            "data": pdf_data
                        }
                    },
                    {
                        "type": "text",
                        "text": "extract text to markdown table"
                    }
                ]
            }
        ],
    )
    return response

In [ ]:
from pprint import pp

pdf_path = "../documents/SDQp2-4Scoring (1).pdf"
pdf_path = "../documents/YP-CORE.pdf"
curr_page = curr_page_pdf(pdf_path, 0)
text_data = extract_text_from_pdf(curr_page)

pp(text_data.content[0].text)

In [ ]:
def markdown_to_json(text_data):
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1024,
        tools=[
            {
                "name": "markdown-to-json",
                "description": "Convert markdown to JSON",
                "input_schema": {
                    "type": "object",
                    "properties": {
                        "headers": {
                            "type": "array",
                            "description": "Column headers of the table",
                            "items": {
                                "type": "string",
                            }
                        },
                        "values": {
                            "type": "array",
                            "description": "Values of the table",
                            "items": {
                                "type": "array",
                                "items": {
                                    "type": "string",
                                }
                            }

                        }
                    },
                    "required": ["headers", "values"],
                },
            }
        ],
        messages=[{"role": "user", "content": text_data}],
    )
    return response

In [ ]:
json_data = markdown_to_json(text_data.content[0].text)
pp(json_data.content[1].input)

In [ ]:
import gspread
import pandas as pd

json_data = json_data.content[1].input
google_credentials = "../google_credentials/document-converter-444316-5d64b96b304e.json"
table_name = "MakeAnImpact_Result"

gc = gspread.service_account(filename=google_credentials)

sh = gc.open(table_name)

# Open a worksheet from spreadsheet with one shot
worksheet = sh.get_worksheet(0)

# convert json to DataFrame
df = pd.DataFrame(json_data['values'], columns=json_data['headers'])

# convert DataFrame to list
values = [df.columns.values.tolist()] + df.values.tolist()

# clear sheet
worksheet.clear()

# upload -> Google Sheets
worksheet.update(values)
